In [1]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [2]:
x = [[1, 2], [3, 4], [5, 6], [7, 8]]
y = [[3], [7], [11], [15]]
X = torch.tensor(x).float()
Y = torch.tensor(y).float()

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
X = X.to(device)
Y = Y.to(device)

In [18]:
class MyDataSet(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x).float()
        self.y = torch.tensor(y).float()
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]

In [19]:
ds = MyDataSet(x, y)

In [20]:
dataloader = DataLoader(ds, batch_size=2, shuffle=True)

In [23]:
for x, y in dataloader:
    print(x, y)

tensor([[3., 4.],
        [5., 6.]]) tensor([[ 7.],
        [11.]])
tensor([[7., 8.],
        [1., 2.]]) tensor([[15.],
        [ 3.]])


In [24]:
class MyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.inputTohidden = nn.Linear(2, 8)
        self.hiddenTooutput = nn.Linear(8, 1)
        self.activation = nn.ReLU()
        
    def forward(self, x):
        x = self.inputTohidden(x)
        x = self.activation(x)
        
        x = self.hiddenTooutput(x)
        return x

In [31]:
net = MyNet().to(device)
criterion = nn.MSELoss()
from torch.optim import SGD
opt = SGD(net.parameters(), lr=0.0001)

In [32]:
import time
loss_history = []
start = time.time()
for i in range(50):
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        
        opt.zero_grad()
        pred = net(x)
        loss_val = criterion(pred, y)
        loss_val.backward()
        opt.step()
        loss_history.append(loss_val.cpu().detach().numpy())
end = time.time()
print(end-start)        

0.17523932456970215


### Predicting on new data points

In [35]:
val_x = [[9, 10]]
val_x = torch.tensor(val_x).float().to(device)

In [36]:
net(val_x)

tensor([[14.9984]], device='cuda:0', grad_fn=<AddmmBackward0>)